In [ ]:

%matplotlib auto


In [ ]:
import os
import inspect
import datetime
from datetime import date, timedelta
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from matplotlib.widgets import Button, SpanSelector, RadioButtons
from matplotlib.dates import num2date, DayLocator


In [ ]:
class Presenter():
    
    def __init__(self, model, view, files):
        self.model = model
        self.view = view
        self.files = files
        
    def showPlots(self):
        self.cleanedDf, self.wholeDf = self.model.parseFiles(self.files)
        self.view.showPlots(self.cleanedDf)
    
    def showDataframe(self):
        return self.wholeDf


In [ ]:
class otpParser():
    
    def __init__(self):
        self.headers = ['accountNr', 'T/J', 'sum', 'currency', 'date', 'date2', 'balance', 'noIdea',
                   'noIdea2', 'comment', 'noIdea3', 'noIdea4', 'comment2']
        
    def parseFiles(self, files):
        try:
            dataFrames = [pd.read_csv(file,  header=None) for file in files]
        except :
            dataFrames = [pd.read_csv(file, sep=';', header=None) for file in files]
        mergedFrame = pd.concat(dataFrames)
#         try:
#             mergedFrame.drop(mergedFrame.columns[[13, 14]], axis=1, inplace=True)
#         except:
#             pass
        mergedFrame.columns = self.headers                                        
        mergedFrame = mergedFrame.reset_index(drop=True)
        mergedFrame = mergedFrame.sort_values(by='date')
        mergedFrame['comment'] = mergedFrame['comment'].replace(np.nan, '', regex=True)
        mergedFrame['comment'] = mergedFrame['comment'].apply((lambda x: ' '.join(x.split())))
        mergedFrame.loc[mergedFrame['comment']=='','comment'] = mergedFrame.loc[(mergedFrame["comment"] == '') , "comment2"]
        cleanDf = mergedFrame.loc[:,['sum', 'date', 'comment','balance']]
        return cleanDf, mergedFrame

    def printDf(self):
        return (self.mergedFrame.head())



In [ ]:

# folder='matyi'
# folder='reka'
headers = ['accountNr', 'T/J', 'sum', 'currency', 'date', 'date2', 'balance', 'noIdea',
                   'noIdea2', 'comment', 'noIdea3', 'noIdea4', 'comment2']
# files =[os.path.join(folder,file) for file in os.listdir('matyi') if file.lower().endswith('csv')]
model = otpParser()
# df1, df=model.parseFiles(files)



In [ ]:
RED = (0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0)
DARK_RED = (0.49803921568627452, 0.12156862745098039, 0.12156862745098039, 1.0)
GREY = (0.5019607843137255, 0.5450980392156862, 0.5882352941176471, 1)
GREEN = (0.098039215686274508, 0.43529411764705883, 0.23921568627450981, 1.0)
DARK_GREEN = (0.0078431372549019607, 0.25490196078431371, 0.0078431372549019607, 1.0)
PURPLE = '#aaace2' # Color of the buttons, titles
LIGHT_GREEN =  (0.1568627450980392, 0.7058823529411765, 0.38823529411764707, 1)
LIGHT_RED = (1.0, 0.2784313725490196, 0.2784313725490196, 1)

LIGHT_GREY = '#d5d8dc'
width = 0.3
WIDTH = 12
G_RAT = (1 + 5 ** 0.5) / 2 # golden ratio
LABEL_ROTATION = 15 # DEGREES
DATEFORMATSTRING = '%Y-%m-%d'
DATEFROMAT = mdates.DateFormatter(DATEFORMATSTRING)
# to highlight recatangles
dark2light={DARK_RED:LIGHT_RED, DARK_GREEN:LIGHT_GREEN}
# to unhighlight recatangles
dark2normal={DARK_RED:RED, DARK_GREEN:GREEN}
light2normal={LIGHT_RED:RED, LIGHT_GREEN:GREEN}

    
class financeViewer():
    
    def __init__(self):

        self.box = dict(facecolor='blue', pad=3, alpha=0.2, boxstyle="Round4,pad=0.3")
        self.testString ="""Date: {}
                            Sum: {} HUF
                            Comment: {}"""
        self.scale1='log'
        self.scale2='log'
        
        self.mode = 'balance' # the other mode is balance mode
        
        self.start, self.end = None, None
    def connect2presenter(self, presenterObject):
        self.presenter = presenterObject
    
    def createFigure(self):

        self.fig = plt.figure(figsize=(WIDTH, WIDTH/G_RAT),facecolor = LIGHT_GREY)

        self.gsp = gridspec.GridSpec(
                nrows = 3, ncols = 2, wspace = 0.05, hspace = 0.45,
                width_ratios = [G_RAT, 1], height_ratios = [(1+G_RAT)/G_RAT, G_RAT, 1])

        self.ax1 = plt.subplot(self.gsp[0,:])
        self.ax2 = plt.subplot(self.gsp[1:,0])
        self.ax3 = plt.subplot(self.gsp[1,1])
        self.ax4 = plt.subplot(self.gsp[2,1])

    def drawAxes(self):

        for ax in [self.ax1,self.ax2,self.ax3, self.ax4]: 
            ax.set_facecolor(GREY)
            
        #####BIG PLOT##       
        self.plotAx1()
        
        ####ZOOM PLOT##
        self.plotAx2()
        
        ##info plot##
        self.txt = self.ax3.text(0.1,0.5,'',
        horizontalalignment='left',
        verticalalignment='center',
        fontsize=12, color='black',
        wrap = True)
        self.ax3.set_xticks([]) 
        self.ax3.set_yticks([]) 
        self.ax3.set_title('info about the transactions', bbox=self.box)

        ### place of buttons##
        self.ax4.set_xticks([]) 
        self.ax4.set_yticks([]) 

    def on_plot_hover(self, event):
#         print ('{} function is called'.format(inspect.stack()[0][3]))
        if not event.inaxes: return
        if event.inaxes!= self.ax2: return

        for idx,bar in enumerate(self.ax2.patches):
            if bar.get_x() < event.xdata < bar.get_x() + bar.get_width():
                if bar.get_y() < event.ydata < bar.get_y() + bar.get_height(): 

                    self.ax2.patches[idx].set_facecolor(dark2light[bar.get_edgecolor()])
                    date_ordinal, y = self.ax2.transData.inverted().transform([event.x, event.y])+0.5
                    
                    # convert the numeric date into a datetime
                    transDate = num2date(date_ordinal).strftime(DATEFORMATSTRING)
                    pdDate = num2date(date_ordinal).strftime('%Y%m%d')
                    try:
                        comment = self.cleanDf.loc[(self.cleanDf['date'] == int(pdDate)) & (abs(self.cleanDf['sum'],)==bar.get_height()),'comment'].iloc[0]
                    except:
                        comment='Record not found'

                    newStr = self.testString.format(transDate,bar.get_height(), comment)
                    self.txt.set_text(newStr)
            else:
                self.ax2.patches[idx].set_facecolor(dark2normal[bar.get_edgecolor()])
        self.fig.canvas.draw()

    def reset_button_on_clicked(self, mouse_event):        
        self.plotAx2()

    def balanceView_button_on_clicked(self, mouse_event):
        self.txt.set_text('Not implemented yet')

    def transView_button_on_clicked(self, mouse_event):
        self.txt.set_text('Not implemented yet')

    def plotAx2(self, startDate=None, endDate=None):  
        self.ax2.cla()
        self.ax2.set_title('Selected duration', bbox=self.box)
        
        
#         if startDate and endDate:
#             currentRange = self.pdRange[startDate:endDate]
            
#             indexes=[]
#             for day in currentRange:
#                 try: 
#                     index=np.where(incomeX==day)[0][0]
#                     indexes.append(index)
#                 except : pass
                
#         else:
        currentRange = self.pdRange
        
        self.ax2.bar(self.incomeX, self.incomeY, color=GREEN,edgecolor=DARK_GREEN)

        baseArray = np.zeros(len(currentRange),dtype=np.float)
        indexes = [np.where(currentRange==day)[0][0] for day in self.incomeX]
        baseArray[indexes] = self.incomeY

        for expenseX, expenseY in zip(self.expenseXs, self.expenseYs):
            ## calculate bottom for this iteration
            currBottomIdxs = [np.where(self.pdRange==day)[0][0] for day in expenseX]
            bottom = baseArray[currBottomIdxs]
            self.ax2.bar(expenseX,expenseY,bottom=bottom, color=RED, edgecolor=DARK_RED)
            ### calculate baseArray for the next iteration

            baseArray[currBottomIdxs] += expenseY   

        
    def plotAx1(self):
        
        self.ax1.cla()
        self.ax1.set_title('Whole duration',bbox=self.box)

        self.ax1.bar(self.incomeX, self.incomeY, color=GREEN,edgecolor=DARK_GREEN)
        
        baseArray = np.zeros(len(self.pdRange),dtype=np.float)
        indexes = [np.where(self.pdRange==day)[0][0] for day in self.incomeX]
        baseArray[indexes] = self.incomeY

        for expenseX, expenseY in zip(self.expenseXs, self.expenseYs):
            ## calculate bottom for this iteration
            currBottomIdxs = [np.where(self.pdRange==day)[0][0] for day in expenseX]
            bottom = baseArray[currBottomIdxs]
            self.ax1.bar(expenseX,expenseY,bottom=bottom, color=RED, edgecolor=DARK_RED)
            ### calculate baseArray for the next iteration

            baseArray[currBottomIdxs] += expenseY            
            
        self.span = SpanSelector(self.ax1, self.onselect, 'horizontal', 
                       rectprops=dict(alpha=0.3, facecolor=RED))            
#         self.ax2.xaxis.set_major_formatter(DATEFROMAT)
#         self.ax2.set_yscale(self.scale2, nonposy='clip')
#         self.ax2.yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
#         plt.setp( self.ax2.xaxis.get_majorticklabels(), rotation=LABEL_ROTATION )
        
    def onselect(self, xmin, xmax):

        dayMin, dayMax = sorted((int(xmin-0.5), int(xmax+0.5)))
        ##xmin, xmax is days from zero, if Xaxis is pandas daterange
        print (dayMin, dayMax)
        yearZero = datetime.datetime.strptime('0001/01/01', "%Y/%m/%d")
        startDate = yearZero + timedelta(days=dayMin)
        endDate = yearZero + timedelta(days=dayMax)
        st=str(startDate)[:10]
        nd=str(endDate)[:10]
        stIdx, = np.where( self.pdRange.values==np.datetime64(st) )
        endIdx, = np.where( self.pdRange.values==np.datetime64(nd) )

        try:
            stIdx , endIdx = stIdx[0], endIdx[0]
        except:
            try:
                stIdx , endIdx = 0, endIdx[0]
            except:
                stIdx , endIdx = stIdx[0], len(self.pdRange)
        print (stIdx, endIdx)
        self.start, self.end = stIdx, endIdx
        self.plotAx2(stIdx, endIdx)
        self.fig.canvas.draw()
        
    def makeButtons(self):

        pos = self.ax4.get_position() # get the  position of axis ,which contains the buttons 
        rowNr, colNr = 1,2
        buttonwidth = 0.14
        buttonheight = 0.08
        Vspace = (pos.width - colNr*buttonwidth)/(colNr+1)
        Hspace = (pos.height - rowNr*buttonheight)/(rowNr+1)
        
        scaleSelectorAx = self.fig.add_axes([pos.x0+Vspace, pos.y0+Hspace, buttonwidth, buttonheight])
        viewSelectorAx = self.fig.add_axes([pos.x0+2*Vspace+buttonwidth, pos.y0+Hspace, buttonwidth, buttonheight])
        axcolor = PURPLE
        self.scaleSelector = RadioButtons(scaleSelectorAx, ('Ax1 linear', 'Ax2 linear', 'Ax1 logaritmic', 'Ax2 logaritmic'))
        self.viewSelector = RadioButtons(viewSelectorAx, ('transaction view', 'balance view'))
        
        for button in [self.scaleSelector, self.viewSelector]:
            for circle in button.circles: # adjust radius here. The default is 0.05
                circle.set_radius(0.08)
   
    def modeButtonClicked(self, label):
            print (label)
            if label == 'balance view':
                self.mode = 'balance'
            elif label == 'transaction view':
                self.mode = 'transaction'
            else: 
                raise ValueError('could not find %s' % label)
            
    def scaleButtonClicked(self, label):

        if label == 'Ax1 linear':
            if self.scale1 == 'linear': return
            self.scale1='linear'
            self.plotAx1()
        elif label == 'Ax2 linear':
            if self.scale2 == 'linear': return
            self.scale2='linear'        
            self.plotAx2()
        elif label =='Ax1 logaritmic':
            if self.scale1 == 'log': return
            self.scale1='log'
            self.plotAx1()
        elif label == 'Ax2 logaritmic':
            if self.scale2 == 'log': return
            self.scale2='log'
            self.plotAx2()
        else: raise ValueError('label not supported')

        self.fig.canvas.draw()
        
    def connectButtons(self):

#         self.reset_button.on_clicked(self.reset_button_on_clicked)
        self.scaleSelector.on_clicked(self.scaleButtonClicked)
        self.viewSelector.on_clicked(self.modeButtonClicked)

      
        
    def calculateAttributes(self):
        self.balance = self.cleanDf['balance'].values
        self.dateAxis = self.cleanDf['date'].values
        self.transactions = self.cleanDf['sum'].values
        self.pdDates = [pd.to_datetime(str(date), format='%Y%m%d') for date in self.dateAxis]

        start = self.pdDates[0]
        end = self.pdDates[-1]

        self.pdRange = pd.date_range(start=start, end=end, periods=None, freq='D', )

        


    def separateTransactions(self):

        values, counts = np.unique(self.pdDates, return_counts=True)
        maxPerDay = max(counts)

        expenseXs, expenseYs = [], []
        incomeX, incomeY = [], []
        smallX, smallY = [], []

        for freq in range(1,max(counts)+1):
            for val, cnt in zip(values, counts):
                if cnt >= freq:
                    index = np.where(np.array(self.pdDates)==val)[0][freq-1]
                    if self.transactions[index] > 0:
                        incomeX.append(val)
                        incomeY.append(self.transactions[index])
                    else:
                        smallX.append(val)
                        smallY.append(-self.transactions[index])

            expenseXs.append(smallX)  
            expenseYs.append(smallY)  
            smallX, smallY = [], []

        self.expenseXs = expenseXs
        self.expenseYs = expenseYs
        self.incomeX = incomeX
        self.incomeY = incomeY
    
    def showPlots(self, cleanDf):
        self.cleanDf = cleanDf
        self.calculateAttributes()
        self.separateTransactions()
        self.createFigure()
        self.drawAxes()

        self.fig.canvas.mpl_connect('button_press_event', self.on_plot_hover) 
        self.fig.subplots_adjust(left=0.06, bottom=0.07, right=0.97, top=0.95)
        self.makeButtons()
        self.connectButtons()

        plt.show()

In [ ]:
# folder='.'
folder='matyi'
# folder='reka'
files =[os.path.join(folder,file) for file in os.listdir(folder) if file.lower().endswith('csv')]
# files =  [file for file in os.listdir(folder) if file.lower().endswith('csv')]
model = otpParser()
view = financeViewer()


myApp = Presenter(model, view, files)

In [ ]:
myApp.showPlots()

In [ ]:
df=myApp.showDataframe()
df

In [ ]:
files

In [ ]:
dfs = []
for file in files:

    try:
        dfs.append(pd.read_csv(file,  header=None))
    except :
        dfs.append(pd.read_csv(file, sep=';', header=None))

for df in dfs:
    print (df.shape)
     
mergedFrame = pd.concat(dfs)
print (mergedFrame.shape)

In [ ]:
headers = ['accountNr', 'T/J', 'sum', 'currency', 'date', 'date2', 'balance', 'noIdea',
                   'noIdea2', 'comment', 'noIdea3', 'noIdea4', 'comment2']
mergedFrame = mergedFrame.reset_index(drop=True)
mergedFrame.drop_duplicates()
mergedFrame.columns = headers
mergedFrame = mergedFrame.sort_values(by='date')

In [ ]:
balance = mergedFrame['balance'].values
dateAxis = mergedFrame['date'].values
transactions = mergedFrame['sum'].values
pdDates = [pd.to_datetime(str(date), format='%Y%m%d') for date in dateAxis]

## BALANCE VIEW
# plt.plot(pdDate,balance, '*')

## TRANSACTION VIEW
values, counts = np.unique(pdDates, return_counts=True)
maxPerDay = max(counts)

expenseXs, expenseYs = [], []
incomeX, incomeY = [], []
smallX, smallY = [], []

for freq in range(1,max(counts)+1):
    for val, cnt in zip(values, counts):
        if cnt >= freq:
            index = np.where(np.array(pdDates)==val)[0][freq-1]
            if transactions[index] > 0:
                incomeX.append(val)
                incomeY.append(transactions[index])
            else:
                smallX.append(val)
                smallY.append(-transactions[index])

    expenseXs.append(smallX)  
    expenseYs.append(smallY)  
    smallX, smallY = [], []

f, ax = plt.subplots()
ax.bar(incomeX, incomeY,color='green',edgecolor='black')

start = pdDate[0]
end = pdDate[-1]

pdRange = pd.date_range(start=start, end=end, periods=None, freq='D', )
lenG = len(pdRange)
baseArray = np.zeros(len(pdRange))
indexes = [np.where(pdRange==day)[0][0] for day in incomeX]
baseArray[indexes] = incomeY

for expenseX, expenseY in zip(expenseXs, expenseYs):
    ## calculate bottom for this iteration
    currBottomIdxs = [np.where(pdRange==day)[0][0] for day in expenseX]
    bottom = baseArray[currBottomIdxs]
    ax.bar(expenseX,expenseY,bottom=bottom, color='red', edgecolor='black')
    ### calculate baseArray for the next iteration

    baseArray[currBottomIdxs] += expenseY


In [ ]:
x1 = [0,1,5,6,10,11]
y1 = [4,2,1,3,2,5]
baseArray = np.zeros(12)
for idx,x in enumerate(x1):
    baseArray[x] +=  y1[idx]

x2 = [0,1,2,4]
y2 = [2,3,2,3]

filteredArray1 = baseArray[x2]

for idx, x, in enumerate(x2):
    baseArray[x] +=  y2[idx]
x3 = [1,2,6,10,9]
y3 = [2,3,2,1,2]
filteredArray2 = baseArray[x3]

print (filteredArray1)
print (filteredArray2)

In [ ]:
f, ax = plt.subplots()
ax.bar(x1,y1, color='red')
ax.bar(x2,y2,bottom=filteredArray1, color='green')
ax.bar(x3,y3,bottom=filteredArray2, color='blue')

In [ ]:
a=4
b=6


In [ ]:
a+b

In [ ]:
import numpy as np
x = np.array([3,5,7,1,9,8,6,6])
y = np.array([2,1,5,10,100,6])

index = np.argsort(x)
print (index)
sorted_x = x[index]
sorted_index = np.searchsorted(sorted_x, y)
print (sorted_index)
yindex = np.take(index, sorted_index, mode="clip")
mask = x[yindex] != y

result = np.ma.array(yindex, mask=mask)
print (result)

In [ ]:
pdrange=np.arange(10)
currange =np.array([1,2,3,4,5])
incomeX = np.array([1,4,5,6,7,9])
incomeY = np.array([2,4,2,4,3,5])

In [ ]:
# indexes = [np.where(incomeX==x)[0][0] for x in currange]
indexes=[]

for x in currange:
    try: 
        index=np.where(incomeX==x)[0][0]
        indexes.append(index)
    except : pass
    


In [ ]:
indexes